<a href="https://colab.research.google.com/github/SilahicAmil/NLP-NLTK/blob/main/Rotten_Tomatoes_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotten Tomatoes Sentiment

https://www.kaggle.com/mrbaloglu/rotten-tomatoes-reviews-dataset

In [1]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Tensorflow
import tensorflow as tf

#Keras
import keras
from keras.layers import Embedding, TextVectorization
from tensorflow.keras import layers
from keras import callbacks

# SkLearn
from sklearn.model_selection import train_test_split

In [2]:
# Unzip data
#!unzip "/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/data_rt.csv (1).zip" -d "/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/"

In [3]:
full_data = pd.read_csv("/content/drive/MyDrive/Rotten_Tomatoes_Sentiment/data_rt.csv")
full_data.head()

,reviews,labels
0,"simplistic , silly and tedious .",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0


In [4]:
full_data.labels.value_counts()

0    5331
1    5331
Name: labels, dtype: int64

# Shuffle Data

In [5]:
full_data = full_data.sample(frac=1, random_state=42)
full_data.head()

,reviews,labels
6830,"this film seems thirsty for reflection , itsel...",1
8600,the movie's thesis -- elegant technology for t...,1
4080,tries too hard to be funny in a way that's too...,0
3079,disturbingly superficial in its approach to th...,0
582,"an ugly , pointless , stupid movie .",0


# Creating Train/Test sets

In [6]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(full_data["reviews"].to_numpy(),
                                                                              full_data["labels"].to_numpy(),
                                                                              test_size=0.12,
                                                                              random_state=42)

In [7]:
len(train_sentences), len(test_sentences)

(9382, 1280)

In [8]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

21.04188872308676

In [9]:
MAX_VOCAB = 10_000
MAX_LEN = 21

txt_vect = TextVectorization(max_tokens=MAX_VOCAB,
                             output_mode="int",
                             output_sequence_length=MAX_LEN)

In [10]:
txt_vect.adapt(train_sentences)

In [11]:
# Most of the work can be done without a GPU since the dataset is small but why not us a GPU for the lulz
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-4eb28e8a-815e-3f89-49a2-6b6cb378cf1c)


# Functions

In [29]:
# Tired of typing compile and fit 

def compile_model(model):
  model.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
  
def fit_model(model, epochs):
  model.fit(train_sentences,
            train_labels,
            epochs=epochs,
            validation_data=(test_sentences, test_labels),
            callbacks=[early_stop])

In [30]:
def acc_loss(model):
  loss, accuracy = model.evaluate(test_sentences, test_labels, verbose=1)
  print(f"Loss: {loss}\nAccuracy: {accuracy}")

# Embeddings Model - Functional API

In [12]:
Embedding = keras.layers.Embedding(input_dim=MAX_VOCAB,
                                   output_dim=128,
                                   input_length=MAX_LEN)

In [ ]:
tf.random.set_seed(42)
#Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

x = txt_vect(inputs)

x = Embedding(x)

# Pooling layer
x = keras.layers.GlobalAveragePooling1D()(x)

# Dense output
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

model_0 = keras.Model(inputs, outputs , name="model_0_functional")

In [ ]:
model_0.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
model_0.summary()

Model: "model_0_functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 21, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,

In [ ]:
hist_0 = model_0.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 10s 28ms/step - loss: 0.6615 - accuracy: 0.6489 - val_loss: 0.6054 - val_accuracy: 0.7125
Epoch 2/5
294/294 [==============================] - 9s 29ms/step - loss: 0.5131 - accuracy: 0.7848 - val_loss: 0.5140 - val_accuracy: 0.7437
Epoch 3/5
294/294 [==============================] - 8s 28ms/step - loss: 0.3822 - accuracy: 0.8567 - val_loss: 0.4956 - val_accuracy: 0.7617
Epoch 4/5
294/294 [==============================] - 7s 23ms/step - loss: 0.2981 - accuracy: 0.8897 - val_loss: 0.4987 - val_accuracy: 0.7469
Epoch 5/5
294/294 [==============================] - 5s 18ms/step - loss: 0.2392 - accuracy: 0.9188 - val_loss: 0.5171 - val_accuracy: 0.7539


## Score - Best

Best score of 75.3%

In [ ]:
loss, accuracy = model_0.evaluate(test_sentences, test_labels)
print(f"Loss: {loss}\nAccuracy: {accuracy*100:2f}")

40/40 [==============================] - 0s 2ms/step - loss: 0.5171 - accuracy: 0.7539
Loss: 0.5171281099319458
Accuracy: 75.390625


# LSTM Model

In [13]:
updated_embeddings = keras.layers.Embedding(input_dim=MAX_VOCAB,
                                           output_dim=128,
                                           embeddings_initializer="uniform",
                                           input_length=MAX_LEN)

In [ ]:
tf.random.set_seed(42)
# LSTM Model
inputs = keras.layers.Input(shape=(1,), dtype="string")

X = txt_vect(inputs)

X = updated_embeddings(X)

#LSTM Layers
X =keras.layers.LSTM(128)(X)
x = keras.layers.Dense(64, activation="relu")(X)

# Dense output
outputs = keras.layers.Dense(1, activation="sigmoid")(X)

model_1 = keras.Model(inputs, outputs, name="LSTM_Model")

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
model_1.summary()

Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
______________________________________________

In [ ]:
hist_2 = model_1.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 17s 49ms/step - loss: 0.5801 - accuracy: 0.6745 - val_loss: 0.5152 - val_accuracy: 0.7484
Epoch 2/5
294/294 [==============================] - 14s 47ms/step - loss: 0.3397 - accuracy: 0.8554 - val_loss: 0.5120 - val_accuracy: 0.7523
Epoch 3/5
294/294 [==============================] - 14s 46ms/step - loss: 0.2054 - accuracy: 0.9219 - val_loss: 0.6275 - val_accuracy: 0.7500
Epoch 4/5
294/294 [==============================] - 14s 47ms/step - loss: 0.1171 - accuracy: 0.9566 - val_loss: 0.7670 - val_accuracy: 0.7469
Epoch 5/5
294/294 [==============================] - 14s 47ms/step - loss: 0.0732 - accuracy: 0.9748 - val_loss: 1.1809 - val_accuracy: 0.7266


## Score

Good score of 72.6%

In [ ]:
loss, accuracy = model_1.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 11ms/step - loss: 1.1809 - accuracy: 0.7266
Loss: 1.1808803081512451
Accuracy: 0.7265625


# RNN Model

In [ ]:
tf.random.set_seed(42)
# Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# RNN
x = keras.layers.SimpleRNN(128)(x)

# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_2 = tf.keras.Model(inputs, outputs, name="RNN_Model")

In [ ]:
model_2.summary()

Model: "RNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_______________________________________________

In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
hist_2 = model_2.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 16s 52ms/step - loss: 0.1515 - accuracy: 0.9480 - val_loss: 0.9997 - val_accuracy: 0.7195
Epoch 2/5
294/294 [==============================] - 15s 52ms/step - loss: 0.0847 - accuracy: 0.9706 - val_loss: 0.8455 - val_accuracy: 0.7328
Epoch 3/5
294/294 [==============================] - 8s 28ms/step - loss: 0.0543 - accuracy: 0.9832 - val_loss: 1.0347 - val_accuracy: 0.7211
Epoch 4/5
294/294 [==============================] - 8s 27ms/step - loss: 0.0404 - accuracy: 0.9873 - val_loss: 1.2837 - val_accuracy: 0.7312
Epoch 5/5
294/294 [==============================] - 8s 27ms/step - loss: 0.0360 - accuracy: 0.9885 - val_loss: 1.3030 - val_accuracy: 0.7383


## Score

Great score of 73.8%

In [ ]:
loss, accuracy = model_2.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 6ms/step - loss: 1.3030 - accuracy: 0.7383
Loss: 1.3029676675796509
Accuracy: 0.73828125


# GRU Model

In [ ]:
tf.random.set_seed(42)

# Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

# Vect
x = txt_vect(inputs)

#Embdedd
x = updated_embeddings(x)

# GRU Cell
x = keras.layers.GRU(128)(x) # for stacking return_sequence=True is needed

# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

#Model creation
model_3 = tf.keras.Model(inputs, outputs, name="Simple_GRU_Model")

In [ ]:
model_3.summary()

Model: "Simple_GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,379,201
Trainable params: 1,379,201
Non-trainable params: 0
________________________________________

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
hist_3 = model_3.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 15s 43ms/step - loss: 0.1300 - accuracy: 0.9468 - val_loss: 0.9905 - val_accuracy: 0.7383
Epoch 2/5
294/294 [==============================] - 12s 42ms/step - loss: 0.0387 - accuracy: 0.9882 - val_loss: 1.1304 - val_accuracy: 0.7383
Epoch 3/5
294/294 [==============================] - 13s 43ms/step - loss: 0.0181 - accuracy: 0.9940 - val_loss: 1.0896 - val_accuracy: 0.7359
Epoch 4/5
294/294 [==============================] - 12s 41ms/step - loss: 0.0168 - accuracy: 0.9945 - val_loss: 1.3645 - val_accuracy: 0.7359
Epoch 5/5
294/294 [==============================] - 13s 44ms/step - loss: 0.0140 - accuracy: 0.9954 - val_loss: 1.5538 - val_accuracy: 0.7437


## Score

Great score overall of 74.3%

In [ ]:
loss, accuracy = model_3.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 10ms/step - loss: 1.5538 - accuracy: 0.7437
Loss: 1.5537843704223633
Accuracy: 0.7437499761581421


In [ ]:
model_3_preds = model_3.predict(test_sentences)
model_3_preds[:15]

array([[6.8500638e-04],
       [3.1536102e-02],
       [9.8600018e-01],
       [2.1265745e-02],
       [4.4313073e-04],
       [1.7069578e-03],
       [9.9998868e-01],
       [9.3506809e-05],
       [9.9999273e-01],
       [9.9997264e-01],
       [9.4307623e-05],
       [9.9941957e-01],
       [9.9930203e-01],
       [1.5103012e-01],
       [5.7750010e-01]], dtype=float32)

In [ ]:
# Squeezing preds so we can get a class
model_3_preds = tf.squeeze(tf.round(model_3_preds))
model_3_preds[:15]

<tf.Tensor: shape=(15,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1.],
      dtype=float32)>

# Conv1D Model

In [ ]:
tf.random.set_seed(42)

#Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# Conv1D layer
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x) # usually you would add another convolutional layer so it can pickup more non linear points in the data. This is jsut a test so not gonna do it LUL

# Pooling layers
x = keras.layers.GlobalMaxPool1D()(x)

# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_4 = tf.keras.Model(inputs, outputs, name="Simple_Conv1D_Model")

In [ ]:
model_4.summary()

Model: "Simple_Conv1D_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 17, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_5 (Dense)             (None, 1)         

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
hist_4 = model_4.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 8s 24ms/step - loss: 0.1618 - accuracy: 0.9508 - val_loss: 0.9507 - val_accuracy: 0.7273
Epoch 2/5
294/294 [==============================] - 7s 23ms/step - loss: 0.0522 - accuracy: 0.9837 - val_loss: 1.2328 - val_accuracy: 0.7258
Epoch 3/5
294/294 [==============================] - 7s 22ms/step - loss: 0.0189 - accuracy: 0.9954 - val_loss: 1.4937 - val_accuracy: 0.7312
Epoch 4/5
294/294 [==============================] - 7s 23ms/step - loss: 0.0073 - accuracy: 0.9988 - val_loss: 1.7188 - val_accuracy: 0.7234
Epoch 5/5
294/294 [==============================] - 7s 23ms/step - loss: 0.0038 - accuracy: 0.9994 - val_loss: 1.8555 - val_accuracy: 0.7273


## Score

Pretty good score of 72.7%

In [ ]:
loss, accuracy = model_4.evaluate(test_sentences, test_labels, verbose=1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

40/40 [==============================] - 0s 5ms/step - loss: 1.8555 - accuracy: 0.7273
Loss: 1.8555233478546143
Accuracy: 0.727343738079071


tired of copy pasting just gonna make a function for accuracy and loss

# Multi-Layer Conv1D Model - GlobalAveragePooling


In [ ]:
tf.random.set_seed(42)
# Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

# Text Vect
X = txt_vect(inputs)

# Embedding
X = updated_embeddings(X)

# Conv1D Layers
X = keras.layers.Conv1D(128, kernel_size=5, activation="relu")(X)
X = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(X)

# Pooling Layer
X = keras.layers.GlobalAveragePooling1D()(X)

# Dense output
outputs = keras.layers.Dense(1, activation="sigmoid")(X)

# Model Creation
model_5 = tf.keras.Model(inputs, outputs, name="MultiLayer_Conv1D_Model")

In [ ]:
model_5.summary()

Model: "MultiLayer_Conv1D_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 conv1d_8 (Conv1D)           (None, 17, 128)           82048     
                                                                 
 conv1d_9 (Conv1D)           (None, 13, 64)            41024     
                                                                 
 global_average_pooling1d_3   (None, 64)               0         
 (GlobalAveragePooling1D)                  

In [ ]:
model_5.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
hist_5 = model_5.fit(train_sentences,
                     train_labels,
                     epochs=5,
                     validation_data=(test_sentences, test_labels))

Epoch 1/5
294/294 [==============================] - 10s 31ms/step - loss: 0.0839 - accuracy: 0.9737 - val_loss: 1.9501 - val_accuracy: 0.7133
Epoch 2/5
294/294 [==============================] - 9s 30ms/step - loss: 0.0062 - accuracy: 0.9990 - val_loss: 3.0653 - val_accuracy: 0.7000
Epoch 3/5
294/294 [==============================] - 9s 30ms/step - loss: 0.0021 - accuracy: 0.9997 - val_loss: 3.4213 - val_accuracy: 0.7117
Epoch 4/5
294/294 [==============================] - 9s 29ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 3.7845 - val_accuracy: 0.7000
Epoch 5/5
294/294 [==============================] - 9s 30ms/step - loss: 8.3808e-04 - accuracy: 0.9999 - val_loss: 4.2591 - val_accuracy: 0.7094


## Score

Okay score of 70.9%

In [ ]:
# Eval
acc_loss(model_5)

40/40 [==============================] - 0s 6ms/step - loss: 4.2591 - accuracy: 0.7094
Loss: 4.259121894836426
Accuracy: 0.7093750238418579


We use a GlobalAveragePool due to logits and labels would need to match for MaxPool.

# MultiLayer GRU Model

In [ ]:
tf.random.set_seed(42)

# Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

# Vect
x = txt_vect(inputs)

#Embdedd
x = updated_embeddings(x)

# GRU Cell
x = keras.layers.GRU(128, activation="relu", return_sequences=True)(x) 
x = keras.layers.GRU(64, activation="relu")(x)


# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

#Model creation
model_8 = tf.keras.Model(inputs, outputs, name="MultiLayer_GRU_Model")

In [ ]:
model_8.summary()

Model: "MultiLayer_GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 gru_7 (GRU)                 (None, 21, 128)           99072     
                                                                 
 gru_8 (GRU)                 (None, 64)                37248     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                              

In [ ]:
compile_model(model_8)

In [ ]:
hist_8 = fit_model(model_8)

Epoch 1/5
294/294 [==============================] - 30s 84ms/step - loss: 0.1175 - accuracy: 0.9516 - val_loss: 1.4300 - val_accuracy: 0.7188
Epoch 2/5
294/294 [==============================] - 17s 59ms/step - loss: 0.0281 - accuracy: 0.9893 - val_loss: 1.9058 - val_accuracy: 0.7234
Epoch 3/5
294/294 [==============================] - 17s 59ms/step - loss: 0.0126 - accuracy: 0.9964 - val_loss: 1.7840 - val_accuracy: 0.7242
Epoch 4/5
294/294 [==============================] - 18s 61ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 2.3645 - val_accuracy: 0.7211
Epoch 5/5
294/294 [==============================] - 22s 74ms/step - loss: 0.0148 - accuracy: 0.9962 - val_loss: 1.8302 - val_accuracy: 0.7180


## Score

Decent score of 71.7%

In [ ]:
#Eval
acc_loss(model_8)

40/40 [==============================] - 1s 13ms/step - loss: 1.8302 - accuracy: 0.7180
Loss: 1.830243468284607
Accuracy: 0.717968761920929


# "Complex" GRU Model w/ Early Stopping and more epochs

In [17]:
# Early stopping

early_stop = keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                        mode="auto", patience=5, 
                                        restore_best_weights=True)

In [ ]:
tf.random.set_seed(42)

# Inputs
inputs = keras.layers.Input(shape=(1,), dtype="string")

# Vect
x = txt_vect(inputs)

#Embdedd
x = updated_embeddings(x)

# GRU Cell
x = keras.layers.GRU(128, return_sequences=True)(x)
x = keras.layers.GRU(64)(x) 

# Dense Layers
x = keras.layers.Dense(64, activation="relu")(x)


# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

#Model creation
model_3_new = tf.keras.Model(inputs, outputs, name="Updated_GRU_Model")

In [ ]:
model_3_new.summary()

Model: "Updated_GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 gru_25 (GRU)                (None, 21, 128)           99072     
                                                                 
 gru_26 (GRU)                (None, 64)                37248     
                                                                 
 dense_23 (Dense)            (None, 64)                4160      
                                                 

In [ ]:
comp_model(model_3_new)

In [ ]:
hist_3_new = model_3_new.fit(train_sentences,
                         train_labels,
                         epochs=50,
                         validation_data=(test_sentences, test_labels),
                         callbacks=[early_stop])

Epoch 1/50
294/294 [==============================] - 23s 64ms/step - loss: 0.0483 - accuracy: 0.9850 - val_loss: 2.2615 - val_accuracy: 0.7148
Epoch 2/50
294/294 [==============================] - 17s 59ms/step - loss: 0.0093 - accuracy: 0.9975 - val_loss: 1.7364 - val_accuracy: 0.7242
Epoch 3/50
294/294 [==============================] - 17s 59ms/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 2.3763 - val_accuracy: 0.7156
Epoch 4/50
294/294 [==============================] - 18s 60ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 2.1551 - val_accuracy: 0.7141
Epoch 5/50
294/294 [==============================] - 17s 59ms/step - loss: 7.4796e-04 - accuracy: 0.9999 - val_loss: 2.7552 - val_accuracy: 0.7219
Epoch 6/50
294/294 [==============================] - 17s 59ms/step - loss: 7.3999e-04 - accuracy: 0.9999 - val_loss: 2.4859 - val_accuracy: 0.7109


## Score

Pretty good score of 72.4%

In [ ]:
acc_loss(model_3_new)

40/40 [==============================] - 1s 13ms/step - loss: 1.7364 - accuracy: 0.7242
Loss: 1.736366629600525
Accuracy: 0.7242187261581421


# Conv1D Model w/ Early Stopping

In [ ]:
tf.random.set_seed(42)

#Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# Conv1D layer
x = keras.layers.Conv1D(64, kernel_size=5,activation="relu")(x)
x = keras.layers.Conv1D(32, kernel_size=5,activation="relu")(x)

# Pooling layers
x = keras.layers.GlobalMaxPool1D()(x)


# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_4_new = tf.keras.Model(inputs, outputs, name="Earlystop_Conv1D_Model")

In [ ]:
model_4_new.summary()

Model: "Earlystop_Conv1D_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 conv1d_23 (Conv1D)          (None, 17, 64)            41024     
                                                                 
 global_max_pooling1d_7 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_28 (Dense)            (None, 1)      

In [ ]:
compile_model(model_4_new)

In [ ]:
hist_4_new = model_4_new.fit(train_sentences,
                         train_labels,
                         epochs=50,
                         validation_data=(test_sentences, test_labels),
                         callbacks=[early_stop])

Epoch 1/50
294/294 [==============================] - 8s 23ms/step - loss: 0.0492 - accuracy: 0.9924 - val_loss: 1.5522 - val_accuracy: 0.7211
Epoch 2/50
294/294 [==============================] - 7s 22ms/step - loss: 0.0077 - accuracy: 0.9981 - val_loss: 1.9184 - val_accuracy: 0.7219
Epoch 3/50
294/294 [==============================] - 7s 22ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 2.1700 - val_accuracy: 0.7195
Epoch 4/50
294/294 [==============================] - 7s 23ms/step - loss: 0.0010 - accuracy: 0.9999 - val_loss: 2.3179 - val_accuracy: 0.7234
Epoch 5/50
294/294 [==============================] - 7s 22ms/step - loss: 7.1295e-04 - accuracy: 0.9999 - val_loss: 2.4450 - val_accuracy: 0.7219
Epoch 6/50
294/294 [==============================] - 6s 22ms/step - loss: 5.2174e-04 - accuracy: 0.9999 - val_loss: 2.5746 - val_accuracy: 0.7211
Epoch 7/50
294/294 [==============================] - 6s 22ms/step - loss: 4.0703e-04 - accuracy: 0.9999 - val_loss: 2.6617 - val_accu

## Score

Pretty good score of 72.5%

In [ ]:
acc_loss(model_4_new)

40/40 [==============================] - 0s 4ms/step - loss: 2.6617 - accuracy: 0.7258
Loss: 2.661738872528076
Accuracy: 0.725781261920929


# Conv1D w/ Dropout Layers

In [20]:
tf.random.set_seed(42)

#Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# Conv1D layer
x = keras.layers.Conv1D(128, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.25)(x)
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x)

# Pooling layers
x = keras.layers.GlobalMaxPool1D()(x)


# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_9 = tf.keras.Model(inputs, outputs, name="Conv1D_Dropout_Model")

In [21]:
compile_model(model_9)

In [22]:
model_9.summary()

Model: "Conv1D_Dropout_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 21)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 21, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 17, 128)           82048     
                                                                 
 dropout_2 (Dropout)         (None, 17, 128)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 13, 64)            41024     
                                              

In [26]:
fit_model(model_9, 50)

Epoch 1/50
294/294 [==============================] - 2s 8ms/step - loss: 0.0179 - accuracy: 0.9938 - val_loss: 1.8166 - val_accuracy: 0.7375
Epoch 2/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0216 - accuracy: 0.9932 - val_loss: 1.9099 - val_accuracy: 0.7422
Epoch 3/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0147 - accuracy: 0.9959 - val_loss: 1.8167 - val_accuracy: 0.7391
Epoch 4/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0113 - accuracy: 0.9964 - val_loss: 2.4097 - val_accuracy: 0.7156
Epoch 5/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0086 - accuracy: 0.9967 - val_loss: 2.5332 - val_accuracy: 0.7094
Epoch 6/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0211 - accuracy: 0.9946 - val_loss: 2.1423 - val_accuracy: 0.7117
Epoch 7/50
294/294 [==============================] - 2s 6ms/step - loss: 0.0278 - accuracy: 0.9913 - val_loss: 1.8935 - val_accuracy: 0.7180


## Score

Great score overall. Only 1% less than the embedding model

In [27]:
acc_loss(model_9)

40/40 [==============================] - 0s 3ms/step - loss: 1.9099 - accuracy: 0.7422
Loss: 1.9098777770996094
Accuracy: 0.7421875


# Conv1D w/ More Dropout and Conv1D Layers

In [47]:
tf.random.set_seed(42)

#Inputs
inputs = keras.layers.Input(shape=(1, ), dtype="string")

#Vect
x = txt_vect(inputs)

# Embedding
x = updated_embeddings(x)

# Conv1D layer
x = keras.layers.Conv1D(128, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.25)(x)
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.15)(x)

# More Conv1D layers
x = keras.layers.Conv1D(128, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Conv1D(64, kernel_size=5, activation="relu")(x)
x = keras.layers.Dropout(0.05)(x)


# Final Pooling Layer
x = keras.layers.GlobalAveragePooling1D()(x)


# Outputs
outputs = keras.layers.Dense(1, activation="sigmoid")(x)

# Model creation
model_10 = tf.keras.Model(inputs, outputs, name="Conv1D_Dropout_Model")

In [48]:
compile_model(model_10)

In [49]:
hist_10 = fit_model(model_10, 50)

Epoch 1/50
294/294 [==============================] - 4s 11ms/step - loss: 0.1390 - accuracy: 0.9459 - val_loss: 1.2497 - val_accuracy: 0.7336
Epoch 2/50
294/294 [==============================] - 3s 9ms/step - loss: 0.0282 - accuracy: 0.9903 - val_loss: 1.6880 - val_accuracy: 0.7352
Epoch 3/50
294/294 [==============================] - 3s 11ms/step - loss: 0.0243 - accuracy: 0.9930 - val_loss: 2.1889 - val_accuracy: 0.7227
Epoch 4/50
294/294 [==============================] - 2s 8ms/step - loss: 0.0201 - accuracy: 0.9941 - val_loss: 1.7931 - val_accuracy: 0.7227
Epoch 5/50
294/294 [==============================] - 3s 9ms/step - loss: 0.0163 - accuracy: 0.9952 - val_loss: 2.0041 - val_accuracy: 0.7242
Epoch 6/50
294/294 [==============================] - 3s 9ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 2.9243 - val_accuracy: 0.7352
Epoch 7/50
294/294 [==============================] - 2s 7ms/step - loss: 0.0101 - accuracy: 0.9975 - val_loss: 2.4696 - val_accuracy: 0.7211


## Score

Looks like more layers made it perform worse with 73.5%

In [50]:
acc_loss(model_10)

40/40 [==============================] - 0s 6ms/step - loss: 1.6880 - accuracy: 0.7352
Loss: 1.6879689693450928
Accuracy: 0.735156238079071
